## 1. Setup Instructions

### Install Dependencies
To get started, install the necessary dependencies using the `requirements.txt` file provided.

**Instructions:**
1. Make sure you have Python installed.
2. Run `pip install -r requirements.txt` in your terminal.
3. Go to https://notabc.app/abc-converter/ to listen to your music
This will install all the following required packages:
- `comet_ml` for experiment tracking
- `tensorflow` for deep learning
- `mitdeeplearning` package for loading datasets and utilities
- `matplotlib`, `opencv-python`, `scipy`, and `tqdm` for visualization and processing tasks

After installing the dependencies, you can proceed with the notebook below.


In [13]:
## 1. Install Dependencies and Import Packages

# Install required libraries
%pip install comet_ml tensorflow mitdeeplearning matplotlib opencv-python scipy --quiet

# Import necessary packages
import comet_ml
import tensorflow as tf
import mitdeeplearning as mdl
import numpy as np
import os
import time
from tqdm import tqdm
from scipy.io.wavfile import write
from IPython import display as ipythondisplay

# Enter your Comet API Key here (required for tracking)
COMET_API_KEY = "YOUR_COMET_API_KEY_HERE"
assert COMET_API_KEY != "", "Please insert your Comet API Key"



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
## 2. Load and Inspect Dataset

# Load the dataset of Irish folk songs
songs = mdl.lab1.load_training_data()

# Print an example song to inspect
print("\nExample song:")
print(songs[0])

# Join the songs into a single string and find unique characters
songs_joined = "\n\n".join(songs)
vocab = sorted(set(songs_joined))
print(f"There are {len(vocab)} unique characters in the dataset.")



Found 817 songs in text

Example song:
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:D Major
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|!
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|!
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|!
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|!
There are 83 unique characters in the dataset.


In [15]:
## 3. Vectorize the Dataset

# Create a character to index and an index to character mapping
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)


# Function to convert songs into numeric representation
def vectorize_string(text):
    return np.array([char2idx[char] for char in text])


vectorized_songs = vectorize_string(songs_joined)

# Check the first few characters and their corresponding vectorized form
print(f"{songs_joined[:10]} ---- characters mapped to int ----> {vectorized_songs[:10]}")


X:1
T:Alex ---- characters mapped to int ----> [49 22 13  0 45 22 26 67 60 79]


In [16]:
## 4. Create Training Examples and Targets

# Function to create training batches
def get_batch(data, seq_length, batch_size):
    n = len(data) - 1
    idx = np.random.choice(n - seq_length, batch_size)
    input_batch = [data[i: i + seq_length] for i in idx]
    output_batch = [data[i + 1: i + seq_length + 1] for i in idx]
    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])
    return x_batch, y_batch


# Test batch function
x_batch, y_batch = get_batch(vectorized_songs, seq_length=10, batch_size=2)
print("Input example:", x_batch[0])
print("Target example:", y_batch[0])


Input example: [62 82 61 59 26 32  1 31 32 26]
Target example: [82 61 59 26 32  1 31 32 26 58]


In [17]:
from tensorflow.keras import layers

# Define the RNN model structure using an Input layer
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, stateful=False):
    model = tf.keras.Sequential([
        layers.Input(shape=(None,), batch_size=batch_size),  # Explicit input layer
        layers.Embedding(vocab_size, embedding_dim),
        layers.LSTM(rnn_units, return_sequences=True, stateful=stateful, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size)
    ])
    return model

# Instantiate the model
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
batch_size = 64

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (64, None, 256)        │        21,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (64, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (64, None, 83)         │        85,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,353,299 (20.42 MB)

 Trainable params: 5,353,299 (20.42 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
import os

def compute_loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
# Check if the trained model exists
checkpoint_path = './training_checkpoints/full_model.keras'

if os.path.exists(checkpoint_path):
    # Load the pre-trained model if it exists
    model = tf.keras.models.load_model(checkpoint_path)
    print("Loaded pre-trained model.")
else:
    # If no checkpoint exists, create and train a new model
    print("No pre-trained model found. Creating and training a new model.")

    # Create the model
    vocab_size = len(vocab)
    embedding_dim = 256
    rnn_units = 1024
    batch_size = 64

    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

    # Compile the model with optimizer
    optimizer = tf.keras.optimizers.Adam()

    # Training step function
    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            predictions = model(x)
            loss = compute_loss(y, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    # Define training parameters
    epochs = 1
    seq_length = 10
    batch_size = 5
    num_training_iterations = 50

    # Training loop
    for epoch in range(epochs):
        for iteration in range(num_training_iterations):
            x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
            loss = train_step(x_batch, y_batch)
            if iteration % 10 == 0:  # Adjust log frequency for visibility
                print(f"Epoch {epoch}, Iteration {iteration}, Loss: {loss.numpy().mean()}")

    # Save the model after training
    os.makedirs('./training_checkpoints', exist_ok=True)
    model.save(checkpoint_path)
    print("New model trained and saved.")


Loaded pre-trained model.


In [21]:
### Prediction of a generated song ###

def generate_text(model, start_string, generation_length=1000):
    # Convert the start string to numbers (vectorize)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to store the generated text
    text_generated = []

    # Remove reset_states(), not needed here
    # Batch size == 1 for inference

    tqdm._instances.clear()

    for i in tqdm(range(generation_length)):
        # Evaluate the inputs and generate the next character predictions
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a multinomial distribution to sample the next character's index
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Use the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Add the predicted character to the generated text
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text using the loaded model
generated_text = generate_text(model, start_string="X", generation_length=500)
print(generated_text)


100%|██████████| 500/500 [00:08<00:00, 61.25it/s]

Xh=02L DU)K4
>^yB-n3.BCPh.:^HBfdg=edwm-Z8Zr'"<IEG4YtG8--Z](vS!Jh<L8YN0f aW6PdweKHpd!!9S^QQ-Oe1y"^]h.:a9t6t23,xb-!L:1tyRX9M-5c/B2__2ej-=I2c23hM]Z"z,'oh|o5[ ecB4Lfe!Sw9]hN4VNulD2-Rc2AeiHs)4ndQ-f2
4'^CspAFL|!C rn1oXc/3:B]EfzIY 7:PYp4d G^qYEiO-#[.3AJ42v.M14<W_EEnSJAe:=0(,^dqhHh#KnHg-tGE=Xp3ByU]6ynz,!jPSb')4<)tYDNvOw5CDeC9<fq/01uOx /oH)#Oq"G/Tt.botE(i1bX/n^visEn/skFA|CgBd-j(|XHm2-,r=AR=76KrF|'d1v"q5ky
!_|2A]HYl>_0!E^l9:y z8X2^9Y:|xQblj35J !Pt23(NZ] X.efa d7e68bvq:Jz5j^jM_/JKHOf|:Y-|lRk-(c!^FDFTmBb.VHl
